# Part 4: Human-in-the-loop

In [1]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

memory = SqliteSaver.from_conn_string(":memory:")


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

In [2]:
graph = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ actions, if desired.
    # interrupt_after=["tools"]
)

In [3]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}
# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_015gknwt2qxENRPRFhF7Fh5G', 'input': {'query': 'LangGraph Python library for language models'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_015gknwt2qxENRPRFhF7Fh5G)
 Call ID: toolu_015gknwt2qxENRPRFhF7Fh5G
  Args:
    query: LangGraph Python library for language models


In [4]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

In [5]:
existing_message = snapshot.values["messages"][-1]
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph Python library for language models'},
  'id': 'toolu_015gknwt2qxENRPRFhF7Fh5G'}]

In [6]:
# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://pypi.org/project/langgraph/", "content": "Project details\nProject links\nStatistics\nView statistics for this project via Libraries.io, or by using our public dataset on Google BigQuery\nMeta\nLicense: Other/Proprietary License (LangGraph License)\nRequires: Python >=3.9.0, <4.0\nMaintainers\nClassifiers\nRelease history\nRelease notifications |\nRSS feed\n0.0.24\nFeb 8, 2024\n0.0.23\nFeb 4, 2024\n0.0.22\nFeb 4, 2024\n0.0.21\nJan 31, 2024\n0.0.20\nJan 27, 2024\n0.0.19\nJan 23, 2024\n0.0.18\nJan 23, 2024\n0.0.17\nJan 23, 2024\n0.0.16\nJan 21, 2024\n0.0.15\nJan 19, 2024\n0.0.14\nJan 18, 2024\n0.0.13\nJan 17, 2024\n0.0.12\nJan 17, 2024\n0.0.11\nJan 16, 2024\n0.0.10\nJan 9, 2024\n0.0.9\nJan 8, 2024\n0.0.8\nyanked\nJan 8, 2024\nDownload files\nDownload the file for your platform. langgraph 0.0.24\npip install langgraph\nCopy PIP instructions\nReleased:\nFeb 8